Notebook to predict N+1 command where N is 1..4 and command sets can be up to len 5

Construct Graph Function

In [1]:
from graph import Node

def adjust_session(session, frequencies = []):
    if len(session) == 2:
        if session[0] == session[1]:
            session = [session[0]]
            frequencies = [2]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 3:
        if session[0] == session[1] and session[0] == session[2]:
            session = session[:1]
            frequencies = [3]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 4:
        if session[0] == session[2] and session[1] == session[3]:
            session = [session[0], session[1]]
            frequencies = [2, 2]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 5:
        if session[0] == session[1] and session[0] == session[2]:
            session = [session[0], session[3], session[4]]
            frequencies = [3, 1, 1]
        elif session[0] == session[2] and session[1] == session[3]:
            session = [session[0], session[1], session[4]]
            frequencies = [2, 2, 1]
        else:
            frequencies = [1] * len(session)
    else:
        frequencies = [1] * len(session)

    return session, frequencies


def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        session = list(filter(filter_empty, session))

        try: 
            first_cmd = session[0]
        except Exception as inst:
            print(session)
            continue

        program = session[0].split()[0]

        # change this to frequency array of length program
        # and add frequency based on index to it    
        freq_addition = []
        session, freq_addition = adjust_session(session=session)

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=freq_addition[0])
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += freq_addition[0]

        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = freq_addition[0]
        else:
            cur_node.commands[first_cmd] += freq_addition[0]

        for cmd in range(1, len(session)):

            if cmd >= len(freq_addition):
                    print(session, len(session))
                    print(freq_addition, len(freq_addition))

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=freq_addition[cmd])
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += freq_addition[cmd]
            
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = freq_addition[cmd]
            else:
                child_node.commands[session[cmd]] += freq_addition[cmd]
            
            cur_node = child_node
        
    return command_dict

Get Prediction Function

In [2]:
def get_prediction(command_list: list[str], graph, result_size=5, graph_depth=4):
    if len(command_list) == 0:
        return None

    commands = command_list[:graph_depth]
    # should be labeled last command
    last_command = command_list[graph_depth]

    if len(commands) == 0:
        return None, [-1]

    program = commands[0].split()[0]

    frequencies = []
    
    if graph.get(program) is None:
        frequencies.append(-1)
        return None, frequencies
    else:
        node = graph[program]
        frequencies.append(node.frequency)

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
            frequencies.append(node.frequency)
        else:
            frequencies.append(-1)
            return None, frequencies
    
    # last frequency if it has it
    next_program = last_command.split()[0]

    if node.children.get(next_program) is not None:
        frequencies.append(node.children[next_program].frequency)
    else:
        frequencies.append(-1)
    
    # previous command should be command_list[depth-1]
    return node.get_prediction(previous_command=command_list[graph_depth-1], num_to_return=result_size), frequencies

Accuracy Function

In [3]:
from thefuzz import fuzz
from thefuzz import process

from statistics import mean

from tqdm import tqdm

from sklearn.model_selection import train_test_split

import pprint
pp = pprint.PrettyPrinter(indent=4)

def append_list(lst, results, commands, graph_depth):
    lst.append({"Results": results, "Expected": commands[graph_depth], "Command Sequence": commands})

def get_accuracy(command_subsets, fail_list = [], fail_frequencies= [], succeed_list = [], succeed_frequencies = [], correct_5_not_3= [], correct_15_not_5 = [], print_fails=False, graph_depth=-1):
    train_data, test_data = train_test_split(command_subsets, test_size=0.2, train_size=0.8, random_state=42)

    train_graph = construct_graph(train_data)

    # filter out nodes that can't get prediction because not enough length so don't contribute and will cause error
    if (graph_depth != -1):
        test_data = [x for x in test_data if len(x) > graph_depth]
    else:
        test_data = [x for x in test_data if len(x) >= 2]
        
    test_size = len(test_data)

    return_3 = 3
    return_5 = 5
    return_15 = 15

    correct = 0
    correct_with_5 = 0
    correct_with_15 = 0
    first_prediction = 0
    has_prediction = 0
    incorrect = 0
    none_count = 0

    for commands in tqdm(test_data, leave=False):
        results_3, frequencies = get_prediction(commands, train_graph, return_3, graph_depth)
        results_5, _ = get_prediction(commands, train_graph, return_5, graph_depth)
        results_15, _ = get_prediction(commands, train_graph, return_15, graph_depth)

        command_freq = [(commands[i], frequencies[i]) for i in range(len(frequencies))]
        
        prev_correct = correct
        prev5_correct =  correct_with_5

        if results_3 is not None:
            has_prediction += 1

            for i in range(len(results_3)):
                if fuzz.ratio(results_3[i][0], commands[graph_depth]) > 85:
                    correct += 1

                    succeed_frequencies.append(mean(frequencies))

                    if correct <= 100:
                        append_list(succeed_list, results_5, command_freq, graph_depth)

                    if i == 0:
                        first_prediction += 1
                
                    break

            if prev_correct == correct:
                fail_frequencies.append(mean(frequencies))
                incorrect += 1

            for i in range(len(results_5)):
                if fuzz.ratio(results_5[i][0], commands[graph_depth]) > 85:
                    correct_with_5 += 1

                    if correct_with_5 - correct <= 100 and prev_correct == correct:
                        append_list(correct_5_not_3, results_5, command_freq, graph_depth)

                    break

            for i in range(len(results_15)):
                if fuzz.ratio(results_15[i][0], commands[graph_depth]) > 85:
                    correct_with_15 += 1

                    if correct_with_15 - correct <= 100 and prev5_correct == correct_with_5:
                        append_list(correct_15_not_5, results_15, command_freq, graph_depth)

                    break

        else:
            none_count += 1

        if results_5 is not None and prev_correct == correct and print_fails:
            append_list(fail_list, results_5, command_freq, graph_depth)  
                              
    return 'Correct Proportion: {:.2f}% |\n Correct in 5 not 3: {:.2f}% |\n Correct in 15 not 5: {:.2f}% |\n Has Prediction and is Correct: {:.2f}% |\n Incorrect Proportion: {:.2f}% |\n None Proportion: {:.2f}% |\n First Prediction: {:.2f}% |\n Average Success Freq: {:.2f} |\n Average Fail Freq: {:.2f}'.format(100 * correct/(test_size-none_count), 100 * correct_with_5/(test_size-none_count), 100 * correct_with_15/(test_size-none_count), 100 * correct/has_prediction, 100 * incorrect/test_size, 100 * none_count/test_size, 100 * first_prediction/test_size, mean(succeed_frequencies)/len(train_data), mean(fail_frequencies)/len(train_data))


In [4]:
from parse import Parser

parser = Parser()

100%|██████████| 56/56 [00:06<00:00,  8.43it/s]


All Commands Parse and Replace Args

In [5]:
subset_size = 5

science_session = parser.parse_commands_per_session(parser.scientists_files)
science_session = parser.parse_commands_into_subsets_sliding_window(science_session, subset_size)
science_session = parser.replace_args_nested(science_session)

experienced_session = parser.parse_commands_per_session(parser.experienced_files)
experienced_session = parser.parse_commands_into_subsets_sliding_window(experienced_session, subset_size)
experienced_session = parser.replace_args_nested(experienced_session)

non_session = parser.parse_commands_per_session(parser.non_programmers_files)
non_session = parser.parse_commands_into_subsets_sliding_window(non_session, subset_size)
non_session = parser.replace_args_nested(non_session)

novice_session = parser.parse_commands_per_session(parser.novice_files)
novice_session = parser.parse_commands_into_subsets_sliding_window(novice_session, subset_size)
novice_session = parser.replace_args_nested(novice_session)

100%|██████████| 56/56 [00:07<00:00,  7.72it/s]


Predicting last command from 1..n-1 commands

In [6]:
i = -1

print("Scientist session:\n", get_accuracy(science_session, graph_depth=i), '\n')
print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i), '\n')
print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i), '\n')
print("Novice session:\n", get_accuracy(novice_session, graph_depth=i), '\n')

Scientist session:
 Correct Proportion: 33.40% |
 Correct in 5 not 3: 39.99% |
 Correct in 15 not 5: 54.49% |
 Has Prediction and is Correct: 33.40% |
 Incorrect Proportion: 64.41% |
 None Proportion: 3.27% |
 First Prediction: 20.32% |
 Average Success Freq: 0.02 |
 Average Fail Freq: 0.02 



Experienced session:
 Correct Proportion: 34.89% |
 Correct in 5 not 3: 41.38% |
 Correct in 15 not 5: 55.91% |
 Has Prediction and is Correct: 34.89% |
 Incorrect Proportion: 62.95% |
 None Proportion: 3.31% |
 First Prediction: 20.94% |
 Average Success Freq: 0.03 |
 Average Fail Freq: 0.05 



Non programmer session:
 Correct Proportion: 33.66% |
 Correct in 5 not 3: 41.04% |
 Correct in 15 not 5: 56.42% |
 Has Prediction and is Correct: 33.66% |
 Incorrect Proportion: 65.10% |
 None Proportion: 1.87% |
 First Prediction: 20.20% |
 Average Success Freq: 0.10 |
 Average Fail Freq: 0.16 



Novice session:
 Correct Proportion: 60.07% |
 Correct in 5 not 3: 65.32% |
 Correct in 15 not 5: 74.78% |
 Has Prediction and is Correct: 60.07% |
 Incorrect Proportion: 39.32% |
 None Proportion: 1.51% |
 First Prediction: 37.58% |
 Average Success Freq: 0.06 |
 Average Fail Freq: 0.06 



Predicting N from 1..N-1 for all

In [7]:
i = -1

all = science_session + experienced_session + non_session + novice_session

print(get_accuracy(all, graph_depth=i))

Correct Proportion: 51.39% |
 Correct in 5 not 3: 57.76% |
 Correct in 15 not 5: 71.88% |
 Has Prediction and is Correct: 51.39% |
 Incorrect Proportion: 48.40% |
 None Proportion: 0.43% |
 First Prediction: 34.36% |
 Average Success Freq: 0.02 |
 Average Fail Freq: 0.03


Predicting for all

In [8]:
all = science_session + experienced_session + non_session + novice_session

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print(get_accuracy(all, graph_depth=i), '\n')

Accuracy predicting command 2 from commands 1 through 1:



Correct Proportion: 31.74% |
 Correct in 5 not 3: 37.24% |
 Correct in 15 not 5: 55.11% |
 Has Prediction and is Correct: 31.74% |
 Incorrect Proportion: 68.26% |
 None Proportion: 0.00% |
 First Prediction: 19.91% |
 Average Success Freq: 0.04 |
 Average Fail Freq: 0.05 

Accuracy predicting command 3 from commands 1 through 2:



Correct Proportion: 46.98% |
 Correct in 5 not 3: 54.33% |
 Correct in 15 not 5: 71.99% |
 Has Prediction and is Correct: 46.98% |
 Incorrect Proportion: 53.02% |
 None Proportion: 0.00% |
 First Prediction: 29.12% |
 Average Success Freq: 0.04 |
 Average Fail Freq: 0.06 

Accuracy predicting command 4 from commands 1 through 3:



Correct Proportion: 62.26% |
 Correct in 5 not 3: 70.48% |
 Correct in 15 not 5: 84.24% |
 Has Prediction and is Correct: 62.26% |
 Incorrect Proportion: 37.73% |
 None Proportion: 0.03% |
 First Prediction: 39.42% |
 Average Success Freq: 0.05 |
 Average Fail Freq: 0.06 

Accuracy predicting command 5 from commands 1 through 4:



Correct Proportion: 66.71% |
 Correct in 5 not 3: 71.84% |
 Correct in 15 not 5: 78.27% |
 Has Prediction and is Correct: 66.71% |
 Incorrect Proportion: 32.69% |
 None Proportion: 1.81% |
 First Prediction: 49.53% |
 Average Success Freq: 0.05 |
 Average Fail Freq: 0.06 



Sampling 0.2 of all   

In [9]:
from random import sample 
import math

all = science_session + experienced_session + non_session + novice_session
all = sample(all, math.floor(.2 * len(all)))

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print(get_accuracy(all, graph_depth=i), '\n')

print(get_accuracy(all, graph_depth=-1), '\n')

Accuracy predicting command 2 from commands 1 through 1:



Correct Proportion: 32.16% |
 Correct in 5 not 3: 37.80% |
 Correct in 15 not 5: 55.58% |
 Has Prediction and is Correct: 32.16% |
 Incorrect Proportion: 67.84% |
 None Proportion: 0.00% |
 First Prediction: 20.35% |
 Average Success Freq: 0.26 |
 Average Fail Freq: 0.34 

Accuracy predicting command 3 from commands 1 through 2:



Correct Proportion: 48.39% |
 Correct in 5 not 3: 55.88% |
 Correct in 15 not 5: 73.28% |
 Has Prediction and is Correct: 48.39% |
 Incorrect Proportion: 51.61% |
 None Proportion: 0.00% |
 First Prediction: 30.21% |
 Average Success Freq: 0.27 |
 Average Fail Freq: 0.35 

Accuracy predicting command 4 from commands 1 through 3:



Correct Proportion: 64.31% |
 Correct in 5 not 3: 72.42% |
 Correct in 15 not 5: 85.93% |
 Has Prediction and is Correct: 64.31% |
 Incorrect Proportion: 35.69% |
 None Proportion: 0.01% |
 First Prediction: 42.05% |
 Average Success Freq: 0.27 |
 Average Fail Freq: 0.35 

Accuracy predicting command 5 from commands 1 through 4:



Correct Proportion: 78.00% |
 Correct in 5 not 3: 83.13% |
 Correct in 15 not 5: 89.43% |
 Has Prediction and is Correct: 78.00% |
 Incorrect Proportion: 21.72% |
 None Proportion: 1.27% |
 First Prediction: 60.55% |
 Average Success Freq: 0.27 |
 Average Fail Freq: 0.35 



Correct Proportion: 54.52% |
 Correct in 5 not 3: 61.05% |
 Correct in 15 not 5: 74.86% |
 Has Prediction and is Correct: 54.52% |
 Incorrect Proportion: 45.34% |
 None Proportion: 0.30% |
 First Prediction: 37.42% |
 Average Success Freq: 0.28 |
 Average Fail Freq: 0.37 



Sampling 0.1 of all 

In [10]:
from random import sample 
import math

all = science_session + experienced_session + non_session + novice_session
all = sample(all, math.floor(.1 * len(all)))

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:".format(i+1, 1, i))

    print(get_accuracy(all, graph_depth=i), '\n')

print(get_accuracy(all, graph_depth=-1), '\n')

Accuracy predicting command 2 from commands 1 through 1:


Correct Proportion: 31.44% |
 Correct in 5 not 3: 37.34% |
 Correct in 15 not 5: 55.23% |
 Has Prediction and is Correct: 31.44% |
 Incorrect Proportion: 68.56% |
 None Proportion: 0.00% |
 First Prediction: 19.81% |
 Average Success Freq: 0.59 |
 Average Fail Freq: 0.77 

Accuracy predicting command 3 from commands 1 through 2:


Correct Proportion: 48.92% |
 Correct in 5 not 3: 56.19% |
 Correct in 15 not 5: 73.29% |
 Has Prediction and is Correct: 48.92% |
 Incorrect Proportion: 51.08% |
 None Proportion: 0.00% |
 First Prediction: 30.11% |
 Average Success Freq: 0.60 |
 Average Fail Freq: 0.78 

Accuracy predicting command 4 from commands 1 through 3:


Correct Proportion: 64.68% |
 Correct in 5 not 3: 72.70% |
 Correct in 15 not 5: 86.04% |
 Has Prediction and is Correct: 64.68% |
 Incorrect Proportion: 35.31% |
 None Proportion: 0.01% |
 First Prediction: 41.28% |
 Average Success Freq: 0.60 |
 Average Fail Freq: 0.79 

Accuracy predicting command 5 from commands 1 through 4:


Correct Proportion: 79.07% |
 Correct in 5 not 3: 84.08% |
 Correct in 15 not 5: 90.68% |
 Has Prediction and is Correct: 79.07% |
 Incorrect Proportion: 20.65% |
 None Proportion: 1.36% |
 First Prediction: 61.74% |
 Average Success Freq: 0.60 |
 Average Fail Freq: 0.79 



Correct Proportion: 54.85% |
 Correct in 5 not 3: 61.38% |
 Correct in 15 not 5: 75.46% |
 Has Prediction and is Correct: 54.85% |
 Incorrect Proportion: 45.00% |
 None Proportion: 0.32% |
 First Prediction: 37.52% |
 Average Success Freq: 0.61 |
 Average Fail Freq: 0.80 



~500 command sets

In [11]:
from random import sample 
import math

all = science_session + experienced_session + non_session + novice_session
print(len(all))

all = sample(all, math.floor(.0005 * len(all)))
print(len(all))

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print(get_accuracy(all, graph_depth=i), '\n')

print(get_accuracy(all, graph_depth=-1), '\n')

978861
489
Accuracy predicting command 2 from commands 1 through 1:



Correct Proportion: 34.69% |
 Correct in 5 not 3: 40.82% |
 Correct in 15 not 5: 54.08% |
 Has Prediction and is Correct: 34.69% |
 Incorrect Proportion: 65.31% |
 None Proportion: 0.00% |
 First Prediction: 24.49% |
 Average Success Freq: 122.83 |
 Average Fail Freq: 160.76 

Accuracy predicting command 3 from commands 1 through 2:



Correct Proportion: 53.42% |
 Correct in 5 not 3: 60.27% |
 Correct in 15 not 5: 75.34% |
 Has Prediction and is Correct: 53.42% |
 Incorrect Proportion: 46.58% |
 None Proportion: 0.00% |
 First Prediction: 36.99% |
 Average Success Freq: 122.84 |
 Average Fail Freq: 160.77 

Accuracy predicting command 4 from commands 1 through 3:



Correct Proportion: 56.86% |
 Correct in 5 not 3: 66.67% |
 Correct in 15 not 5: 82.35% |
 Has Prediction and is Correct: 56.86% |
 Incorrect Proportion: 43.14% |
 None Proportion: 0.00% |
 First Prediction: 35.29% |
 Average Success Freq: 122.85 |
 Average Fail Freq: 160.78 

Accuracy predicting command 5 from commands 1 through 4:



Correct Proportion: 70.83% |
 Correct in 5 not 3: 79.17% |
 Correct in 15 not 5: 83.33% |
 Has Prediction and is Correct: 70.83% |
 Incorrect Proportion: 28.00% |
 None Proportion: 4.00% |
 First Prediction: 60.00% |
 Average Success Freq: 122.85 |
 Average Fail Freq: 160.78 



Correct Proportion: 53.61% |
 Correct in 5 not 3: 61.86% |
 Correct in 15 not 5: 72.16% |
 Has Prediction and is Correct: 53.61% |
 Incorrect Proportion: 45.92% |
 None Proportion: 1.02% |
 First Prediction: 39.80% |
 Average Success Freq: 122.87 |
 Average Fail Freq: 160.79 



~100 command sets

In [12]:
from random import sample 
import math

all = science_session + experienced_session + non_session + novice_session
print(len(all))

all = sample(all, math.floor(.0001 * len(all)))
print(len(all))

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print(get_accuracy(all, graph_depth=i), '\n')

print(get_accuracy(all, graph_depth=-1), '\n')

978861
97
Accuracy predicting command 2 from commands 1 through 1:



Correct Proportion: 15.00% |
 Correct in 5 not 3: 20.00% |
 Correct in 15 not 5: 40.00% |
 Has Prediction and is Correct: 15.00% |
 Incorrect Proportion: 85.00% |
 None Proportion: 0.00% |
 First Prediction: 5.00% |
 Average Success Freq: 623.93 |
 Average Fail Freq: 816.52 

Accuracy predicting command 3 from commands 1 through 2:



Correct Proportion: 43.75% |
 Correct in 5 not 3: 50.00% |
 Correct in 15 not 5: 87.50% |
 Has Prediction and is Correct: 43.75% |
 Incorrect Proportion: 56.25% |
 None Proportion: 0.00% |
 First Prediction: 12.50% |
 Average Success Freq: 623.95 |
 Average Fail Freq: 816.53 

Accuracy predicting command 4 from commands 1 through 3:



Correct Proportion: 27.27% |
 Correct in 5 not 3: 54.55% |
 Correct in 15 not 5: 81.82% |
 Has Prediction and is Correct: 27.27% |
 Incorrect Proportion: 72.73% |
 None Proportion: 0.00% |
 First Prediction: 18.18% |
 Average Success Freq: 623.94 |
 Average Fail Freq: 816.53 

Accuracy predicting command 5 from commands 1 through 4:



Correct Proportion: 60.00% |
 Correct in 5 not 3: 80.00% |
 Correct in 15 not 5: 80.00% |
 Has Prediction and is Correct: 60.00% |
 Incorrect Proportion: 40.00% |
 None Proportion: 0.00% |
 First Prediction: 60.00% |
 Average Success Freq: 623.95 |
 Average Fail Freq: 816.53 



Correct Proportion: 30.00% |
 Correct in 5 not 3: 45.00% |
 Correct in 15 not 5: 75.00% |
 Has Prediction and is Correct: 30.00% |
 Incorrect Proportion: 70.00% |
 None Proportion: 0.00% |
 First Prediction: 15.00% |
 Average Success Freq: 623.95 |
 Average Fail Freq: 816.55 



Accuracy predicting N+1 command from N commands with all commands in graph

In [13]:
for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session:\n", get_accuracy(science_session, graph_depth=i), '\n')
    print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i), '\n')
    print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i), '\n')
    print("Novice session:\n", get_accuracy(novice_session, graph_depth=i), '\n')

Accuracy predicting command 2 from commands 1 through 1:



Scientist session:
 Correct Proportion: 23.52% |
 Correct in 5 not 3: 28.62% |
 Correct in 15 not 5: 48.36% |
 Has Prediction and is Correct: 23.52% |
 Incorrect Proportion: 76.48% |
 None Proportion: 0.00% |
 First Prediction: 13.74% |
 Average Success Freq: 0.15 |
 Average Fail Freq: 0.23 



Experienced session:
 Correct Proportion: 24.02% |
 Correct in 5 not 3: 29.67% |
 Correct in 15 not 5: 49.65% |
 Has Prediction and is Correct: 24.02% |
 Incorrect Proportion: 75.98% |
 None Proportion: 0.00% |
 First Prediction: 13.85% |
 Average Success Freq: 0.26 |
 Average Fail Freq: 0.41 



Non programmer session:
 Correct Proportion: 24.16% |
 Correct in 5 not 3: 31.07% |
 Correct in 15 not 5: 51.60% |
 Has Prediction and is Correct: 24.16% |
 Incorrect Proportion: 75.84% |
 None Proportion: 0.00% |
 First Prediction: 15.59% |
 Average Success Freq: 0.82 |
 Average Fail Freq: 1.33 



Novice session:
 Correct Proportion: 55.56% |
 Correct in 5 not 3: 61.64% |
 Correct in 15 not 5: 73.99% |
 Has Prediction and is Correct: 55.56% |
 Incorrect Proportion: 44.44% |
 None Proportion: 0.00% |
 First Prediction: 38.04% |
 Average Success Freq: 0.31 |
 Average Fail Freq: 0.44 

Accuracy predicting command 3 from commands 1 through 2:



...

Novice session:
 Correct Proportion: 73.79% |
 Correct in 5 not 3: 79.94% |
 Correct in 15 not 5: 90.33% |
 Has Prediction and is Correct: 73.79% |
 Incorrect Proportion: 26.21% |
 None Proportion: 0.02% |
 First Prediction: 38.42% |
 Average Success Freq: 0.36 |
 Average Fail Freq: 0.48 

Accuracy predicting command 5 from commands 1 through 4:



Scientist session:
 Correct Proportion: 67.33% |
 Correct in 5 not 3: 72.18% |
 Correct in 15 not 5: 78.77% |
 Has Prediction and is Correct: 67.33% |
 Incorrect Proportion: 32.01% |
 None Proportion: 2.03% |
 First Prediction: 52.05% |
 Average Success Freq: 0.22 |
 Average Fail Freq: 0.30 



Experienced session:
 Correct Proportion: 70.71% |
 Correct in 5 not 3: 75.85% |
 Correct in 15 not 5: 81.84% |
 Has Prediction and is Correct: 70.71% |
 Incorrect Proportion: 28.68% |
 None Proportion: 2.10% |
 First Prediction: 54.39% |
 Average Success Freq: 0.36 |
 Average Fail Freq: 0.49 



Non programmer session:
 Correct Proportion: 63.19% |
 Correct in 5 not 3: 70.76% |
 Correct in 15 not 5: 80.57% |
 Has Prediction and is Correct: 63.19% |
 Incorrect Proportion: 36.39% |
 None Proportion: 1.14% |
 First Prediction: 44.06% |
 Average Success Freq: 1.14 |
 Average Fail Freq: 1.54 



Novice session:
 Correct Proportion: 77.67% |
 Correct in 5 not 3: 81.87% |
 Correct in 15 not 5: 87.07% |
 Has Prediction and is Correct: 77.67% |
 Incorrect Proportion: 22.19% |
 None Proportion: 0.65% |
 First Prediction: 57.84% |
 Average Success Freq: 0.36 |
 Average Fail Freq: 0.49 



Consolidated predicting next command. Graph depth of `-1` should accomplish this.

In [14]:
import os
import csv

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    fail_frequencies = []
    succeed_frequencies = []

    print(get_accuracy(datasets[i], fail_frequencies=fail_frequencies, succeed_frequencies=succeed_frequencies, print_fails=True, graph_depth=-1))

    os.makedirs(os.path.dirname("fails/frequencies_all/" + dataset_names[i] + ".csv"), exist_ok=True)
    with open("fails/frequencies_all/" + dataset_names[i] + ".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fail_frequencies)

    os.makedirs(os.path.dirname("successes/frequencies_all/" + dataset_names[i] + ".csv"), exist_ok=True)
    with open("successes/frequencies_all/" + dataset_names[i] + ".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(succeed_frequencies)


Correct Proportion: 46.72% |
 Correct in 5 not 3: 53.21% |
 Correct in 15 not 5: 68.57% |
 Has Prediction and is Correct: 46.72% |
 Incorrect Proportion: 53.03% |
 None Proportion: 0.48% |
 First Prediction: 31.84% |
 Average Success Freq: 0.33 |
 Average Fail Freq: 0.67


Correct Proportion: 48.75% |
 Correct in 5 not 3: 56.11% |
 Correct in 15 not 5: 71.27% |
 Has Prediction and is Correct: 48.75% |
 Incorrect Proportion: 51.00% |
 None Proportion: 0.49% |
 First Prediction: 33.36% |
 Average Success Freq: 0.50 |
 Average Fail Freq: 1.08


Correct Proportion: 43.79% |
 Correct in 5 not 3: 52.02% |
 Correct in 15 not 5: 68.29% |
 Has Prediction and is Correct: 43.79% |
 Incorrect Proportion: 56.07% |
 None Proportion: 0.26% |
 First Prediction: 28.68% |
 Average Success Freq: 1.85 |
 Average Fail Freq: 3.82


Correct Proportion: 67.72% |
 Correct in 5 not 3: 73.36% |
 Correct in 15 not 5: 83.40% |
 Has Prediction and is Correct: 67.72% |
 Incorrect Proportion: 32.23% |
 None Proportion: 0.16% |
 First Prediction: 44.35% |
 Average Success Freq: 1.14 |
 Average Fail Freq: 1.17


Save files for all data

In [15]:
import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    for j in range (1, 5):
        fail_list = []
        succeed_list = []
        correct_5_not_3 = []
        correct_15_not_5 = []

        get_accuracy(datasets[i], fail_list=fail_list, succeed_list=succeed_list, correct_5_not_3=correct_5_not_3, correct_15_not_5=correct_15_not_5, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(fail_list))

        os.makedirs(os.path.dirname("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(succeed_list))

        os.makedirs(os.path.dirname("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_5_not_3))

        os.makedirs(os.path.dirname("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_15_not_5))


KeyboardInterrupt: 

Save frequencies for successes and failures to CSV

In [ ]:
import os
import csv

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    for j in range (1, 5):
        fail_frequencies = []
        succeed_frequencies = []

        get_accuracy(datasets[i], fail_frequencies=fail_frequencies, succeed_frequencies=succeed_frequencies, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv"), exist_ok=True)
        with open("fails/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv", "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fail_frequencies)

        os.makedirs(os.path.dirname("successes/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv"), exist_ok=True)
        with open("successes/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv", "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(succeed_frequencies)


Sampled Commands Parse and Replace Args

In [ ]:
from random import sample
import math

subset_size = 5

science_session_sample = parser.parse_commands_per_session(sample(parser.scientists_files, math.floor(0.2 * len(parser.scientists_files))))
science_session_sample = parser.parse_commands_into_subsets_sliding_window(science_session_sample, subset_size)
science_session_sample = parser.replace_args_nested(science_session_sample)

experienced_session_sample = parser.parse_commands_per_session(sample(parser.experienced_files, math.floor(0.2 * len(parser.experienced_files))))
experienced_session_sample = parser.parse_commands_into_subsets_sliding_window(experienced_session_sample, subset_size)
experienced_session_sample = parser.replace_args_nested(experienced_session_sample)

non_session_sample = parser.parse_commands_per_session(sample(parser.non_programmers_files, math.floor(0.2 * len(parser.non_programmers_files))))
non_session_sample = parser.parse_commands_into_subsets_sliding_window(non_session_sample, subset_size)
non_session_sample = parser.replace_args_nested(non_session_sample)

novice_session_sample = parser.parse_commands_per_session(sample(parser.novice_files, math.floor(0.2 * len(parser.novice_files))))
novice_session_sample = parser.parse_commands_into_subsets_sliding_window(novice_session_sample, subset_size)
novice_session_sample = parser.replace_args_nested(novice_session_sample)

100%|██████████| 11/11 [00:02<00:00,  3.73it/s]


Accuracy predicting N+1 command from N commands with subset of data

In [ ]:
for i in range(1, 5):
    print("Accuracy Predicting Command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
    print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
    print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
    print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))

Accuracy Predicting Command 2 from commands 1 through 1:



Scientist session sample:
 Correct Proportion: 62.75% |
 Correct in 5 not 3: 69.17% |
 Correct in 15 not 5: 82.97% |
 Has Prediction and is Correct: 62.75% |
 Incorrect Proportion: 37.25% |
 None Proportion: 0.00% |
 First Prediction: 45.99% |
 Average Success Freq: 0.41 |
 Average Fail Freq: 0.74


Experienced session sample:
 Correct Proportion: 65.86% |
 Correct in 5 not 3: 73.60% |
 Correct in 15 not 5: 86.97% |
 Has Prediction and is Correct: 65.86% |
 Incorrect Proportion: 34.14% |
 None Proportion: 0.00% |
 First Prediction: 45.30% |
 Average Success Freq: 0.63 |
 Average Fail Freq: 1.09


Non programmer session sample:
 Correct Proportion: 59.77% |
 Correct in 5 not 3: 68.18% |
 Correct in 15 not 5: 83.48% |
 Has Prediction and is Correct: 59.77% |
 Incorrect Proportion: 40.23% |
 None Proportion: 0.00% |
 First Prediction: 41.34% |
 Average Success Freq: 3.59 |
 Average Fail Freq: 6.32


Novice session sample:
 Correct Proportion: 70.98% |
 Correct in 5 not 3: 77.40% |
 Correct in 15 not 5: 87.91% |
 Has Prediction and is Correct: 70.98% |
 Incorrect Proportion: 29.01% |
 None Proportion: 0.01% |
 First Prediction: 46.98% |
 Average Success Freq: 0.51 |
 Average Fail Freq: 0.86
Accuracy Predicting Command 3 from commands 1 through 2:



...

Novice session sample:
 Correct Proportion: 90.72% |
 Correct in 5 not 3: 93.81% |
 Correct in 15 not 5: 97.99% |
 Has Prediction and is Correct: 90.72% |
 Incorrect Proportion: 9.28% |
 None Proportion: 0.04% |
 First Prediction: 72.93% |
 Average Success Freq: 0.49 |
 Average Fail Freq: 0.85
Accuracy Predicting Command 5 from commands 1 through 4:



Scientist session sample:
 Correct Proportion: 79.13% |
 Correct in 5 not 3: 80.96% |
 Correct in 15 not 5: 83.09% |
 Has Prediction and is Correct: 79.13% |
 Incorrect Proportion: 20.77% |
 None Proportion: 0.50% |
 First Prediction: 69.57% |
 Average Success Freq: 0.44 |
 Average Fail Freq: 0.76


Experienced session sample:
 Correct Proportion: 81.25% |
 Correct in 5 not 3: 82.65% |
 Correct in 15 not 5: 83.65% |
 Has Prediction and is Correct: 81.25% |
 Incorrect Proportion: 18.72% |
 None Proportion: 0.16% |
 First Prediction: 72.56% |
 Average Success Freq: 0.64 |
 Average Fail Freq: 1.09


Non programmer session sample:
 Correct Proportion: 78.09% |
 Correct in 5 not 3: 81.61% |
 Correct in 15 not 5: 85.89% |
 Has Prediction and is Correct: 78.09% |
 Incorrect Proportion: 21.80% |
 None Proportion: 0.50% |
 First Prediction: 63.16% |
 Average Success Freq: 3.63 |
 Average Fail Freq: 6.20


Novice session sample:
 Correct Proportion: 84.80% |
 Correct in 5 not 3: 87.42% |
 Correct in 15 not 5: 90.10% |
 Has Prediction and is Correct: 84.80% |
 Incorrect Proportion: 15.17% |
 None Proportion: 0.18% |
 First Prediction: 69.14% |
 Average Success Freq: 0.49 |
 Average Fail Freq: 0.83


Predict N from 1..n-1 in samples

In [ ]:
i=-1

print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))

Scientist session sample:
 Correct Proportion: 77.59% |
 Correct in 5 not 3: 82.15% |
 Correct in 15 not 5: 89.06% |
 Has Prediction and is Correct: 77.59% |
 Incorrect Proportion: 22.38% |
 None Proportion: 0.12% |
 First Prediction: 63.68% |
 Average Success Freq: 0.35 |
 Average Fail Freq: 0.68


Experienced session sample:
 Correct Proportion: 80.30% |
 Correct in 5 not 3: 84.65% |
 Correct in 15 not 5: 91.14% |
 Has Prediction and is Correct: 80.30% |
 Incorrect Proportion: 19.69% |
 None Proportion: 0.05% |
 First Prediction: 65.05% |
 Average Success Freq: 0.52 |
 Average Fail Freq: 0.95


Non programmer session sample:
 Correct Proportion: 71.33% |
 Correct in 5 not 3: 77.58% |
 Correct in 15 not 5: 88.24% |
 Has Prediction and is Correct: 71.33% |
 Incorrect Proportion: 28.64% |
 None Proportion: 0.11% |
 First Prediction: 54.21% |
 Average Success Freq: 2.95 |
 Average Fail Freq: 5.43


Novice session sample:
 Correct Proportion: 81.20% |
 Correct in 5 not 3: 85.48% |
 Correct in 15 not 5: 91.90% |
 Has Prediction and is Correct: 81.20% |
 Incorrect Proportion: 18.79% |
 None Proportion: 0.06% |
 First Prediction: 62.20% |
 Average Success Freq: 0.40 |
 Average Fail Freq: 0.69


Expand pipes and replace args

In [ ]:
science = parser.filter_commands_with_pipe(parser.scientists_commands)
experienced = parser.filter_commands_with_pipe(parser.experienced_commands)
non = parser.filter_commands_with_pipe(parser.non_programmers_commands)
novice = parser.filter_commands_with_pipe(parser.novice_commands)

science_pipes = parser.expand_piped_commands(science)
experienced_pipes = parser.expand_piped_commands(experienced)
non_pipes = parser.expand_piped_commands(non)
novice_pipes = parser.expand_piped_commands(novice)

science_pipes_expand = parser.replace_arg_expanded_pipe(science_pipes)
experienced_pipes_expand = parser.replace_arg_expanded_pipe(experienced_pipes)
non_pipes_expand = parser.replace_arg_expanded_pipe(non_pipes)
novice_pipes_expand = parser.replace_arg_expanded_pipe(novice_pipes)

In [ ]:
print("Science pipes metrics:\n", get_accuracy(science_pipes_expand), "\n")
print("Experienced pipes metrics:\n", get_accuracy(experienced_pipes_expand), "\n")
print("Non programmers pipes metrics:\n", get_accuracy(non_pipes_expand), "\n")
print("Novice pipes metrics:\n", get_accuracy(novice_pipes_expand), "\n")

[]


Science pipes metrics:
 Correct Proportion: 84.50% |
 Correct in 5 not 3: 87.34% |
 Correct in 15 not 5: 92.36% |
 Has Prediction and is Correct: 84.50% |
 Incorrect Proportion: 14.95% |
 None Proportion: 3.58% |
 First Prediction: 66.95% |
 Average Success Freq: 0.05 |
 Average Fail Freq: 0.06 



Experienced pipes metrics:
 Correct Proportion: 86.98% |
 Correct in 5 not 3: 93.02% |
 Correct in 15 not 5: 95.81% |
 Has Prediction and is Correct: 86.98% |
 Incorrect Proportion: 12.79% |
 None Proportion: 1.83% |
 First Prediction: 74.43% |
 Average Success Freq: 0.15 |
 Average Fail Freq: 0.14 



Non programmers pipes metrics:
 Correct Proportion: 92.01% |
 Correct in 5 not 3: 93.37% |
 Correct in 15 not 5: 96.49% |
 Has Prediction and is Correct: 92.01% |
 Incorrect Proportion: 7.99% |
 None Proportion: 0.00% |
 First Prediction: 77.19% |
 Average Success Freq: 0.24 |
 Average Fail Freq: 0.15 



Novice pipes metrics:
 Correct Proportion: 90.48% |
 Correct in 5 not 3: 92.86% |
 Correct in 15 not 5: 95.24% |
 Has Prediction and is Correct: 90.48% |
 Incorrect Proportion: 9.09% |
 None Proportion: 4.55% |
 First Prediction: 52.27% |
 Average Success Freq: 2.82 |
 Average Fail Freq: 1.77 

